In [21]:
!pip install transformers sentence-transformers datasets rouge-score streamlit evaluate

In [22]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import evaluate
import numpy as np
import streamlit as st

In [23]:
dataset= []

with open('/content/AI_ethics_Dataset.txt', 'r') as file:
  dataset= file.readlines()
  print(f'Loaded {len(dataset)} lines.')

Loaded 50 lines.


In [24]:
Embedding_model = 'all-MiniLM-L6-v2'
embedder = SentenceTransformer(Embedding_model)

Vector_DB= []
for chunk in dataset:
  embedded = embedder.encode([chunk])[0]
  Vector_DB.append([chunk, embedded]) # Store both chunk and embedding
  print(f'Embedded lines as (shape: {embedded.shape})')

Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines as (shape: (384,))
Embedded lines

In [25]:
def cosine_similarity(a, b):
  dot_prod= np.dot(a, b)
  norm_a= np.linalg.norm(a)
  norm_b= np.linalg.norm(b)
  return dot_prod / norm_a * norm_b

In [26]:
def retrieve(query, top_n= 3):
  query_embedding= embedder.encode([query])[0]
  similarities= []
  for chunk, embedding in Vector_DB:
    sim= cosine_similarity(query_embedding, embedding)
    similarities.append([chunk, sim])
    similarities.sort(key= lambda x: x[1], reverse= True)
    return similarities[:top_n]

In [27]:
Generation_model= 'gpt2'
generator= pipeline('text-generation', model= Generation_model)

def generate_ans(query, retrieved_knowledge):
  context= '\n'.join(f'- {chunk}' for chunk, _ in retrieved_knowledge)
  prompt= f'''You are a helpful AI ethics expert.
  Use ONLY the following context to answer the question. Do NOT use any external knowledge. Be concise and accurate.

  context: {context}
  Question: {query}
  Answer:'''

  response= generator(prompt, max_new_tokens= 50, num_return_sequences= 1, do_sample= True, temperature= 0.7)[0]['generated_text']
  answer= response.split('Answer:')[-1].strip()
  return answer

Device set to use cpu


In [28]:
input_query= input('Ask a quetion about AI ethics: ')
retrieved= retrieve(input_query)
print('Retrieved Knowledge: ')
for chunk, sim in retrieved:
  print(f'- (Similarity: {sim:.2f}) {chunk}')

answer= generate_ans(input_query, retrieved)
print('Generated answer: ', answer)

Ask a quetion about AI ethics: what is ai ethics


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Retrieved Knowledge: 
- (Similarity: 0.91) AI ethics is the study of moral principles guiding the design, use, and impact of artificial intelligence.

Generated answer:  "Ai ethics is a survey of human behavior, including ethical principles (e.g., ethical rules of behavior) that are applied in practice to the behavior of the human social and economic system. This is a survey of human behavior with respect


In [29]:
rouge_metric= evaluate.load('rouge')
input_set= [
  ("What is AI ethics?", "AI ethics is the study of moral principles guiding the design, use, and impact of artificial intelligence."),
  ("What does fairness ensure in AI?", "Fairness ensures AI systems do not discriminate against individuals or groups."),
  ("What causes algorithmic bias?", "Algorithmic bias occurs when AI produces unfair outcomes due to skewed training data or flawed design."),
  ("How can bias enter AI systems?", "Bias can enter through biased datasets, human labeling errors, or unbalanced model training."),
  ("What is transparency in AI?", "Transparency means making AI decision processes understandable to humans."),
  ("Why is explainability important?", "Explainability helps users trust AI by showing how decisions are made."),
  ("What does accountability ensure?", "Accountability ensures humans remain responsible for AI outcomes, not the machines."),
  ("What is data privacy?", "Data privacy means protecting personal information used in AI training and deployment."),
  ("What does GDPR require?", "GDPR requires AI systems to handle personal data lawfully, fairly, and transparently."),
  ("What is informed consent?", "Informed consent means users agree to how their data is collected and used.")
]
scores= []
for q, ref in input_set:
  ret= retrieve(q)
  ans= generate_ans(q, ret)
  score= rouge_metric.compute(predictions= [ans], references= [ref])['rougeL']
  scores.append(score)
print(f'Avg score is {np.mean(scores):.2f}.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Avg score is 0.13.


In [30]:
def streamlit_app():
  st.title('RAG Ethics Chatbot')
  query= st.text_input('Ask AI ethics question: ')
  if query:
    ret= retrieve(query)
    ans= generate_ans(query, ret)
    st.write('Retrieved: ', [c for c, _ in ret])
    st.write('Answer: ', ans)

if __name__ == '__main__':
  streamlit_app()

2025-10-21 10:36:05.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 10:36:05.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar